In [1]:
import pandas as pd
import json
import os
import glob

In [2]:
with open ('../raw-files/all-party-information.json', 'r') as f:
    parties = json.load(f)['data_list']

partiesDict = dict(map(lambda x: (x['id_dga'], x), parties))

def get_partyid_for_image(dgaId):
    return partiesDict[dgaId]['id']

In [3]:
get_partyid_for_image(192) == 81

True

In [4]:
def img_path(*elements):
    return "http://202.6.26.16/images/candidate/thumb/%s/%s/%s.jpg" % elements

In [5]:
df_zone_candidates = pd.read_csv('../raw-files/candidates.csv')
df_pl_candidates = pd.read_csv('../data/partylist_candidates.csv')

In [6]:
df_zone_candidates['image'] = df_zone_candidates.apply(lambda x: img_path(x['province_id'], x['zone_number'], x['CandidateId']), axis=1)
df_zone_candidates['name'] = df_zone_candidates.apply(lambda r: '%s %s' % (r['FirstName'], r['LastName']), axis=1)
df_zone_candidates['desc'] = df_zone_candidates.apply(lambda r: 'ส.ส. เขต (%s เขต %s)' % (r['province_name'], r['zone_number']), axis=1)
df_zone_candidates['memberType'] = 'zone'
df_zone_candidates['PartyName'] = df_zone_candidates.PartyName.apply(lambda x: x.strip())

In [7]:
df_pl_candidates['name'] = df_pl_candidates.apply(lambda x: '%s %s' % (x['FirstName'], x['LastName']), axis=1)
df_pl_candidates['desc'] = df_pl_candidates.apply(lambda x: 'ส.ส. บัญชีรายชื่อ (ลำดับที่ %s)' % (x['CandidateNo']), axis=1)
df_pl_candidates['image'] = df_pl_candidates.apply(lambda x: img_path('00', get_partyid_for_image(x['PartyNumber']), x['CandidateId']), axis=1)
df_pl_candidates['memberType'] = 'partylist'
df_pl_candidates['zone_number'] = "-"
df_pl_candidates['province_name'] = "-"
df_pl_candidates['PartyName'] = df_pl_candidates.PartyName.apply(lambda x: x.strip())

In [8]:
'We have %d ส.ส. ทั้งหมด, (region %d, partylist %d)'%(
    len(df_pl_candidates) + len(df_zone_candidates),
    len(df_zone_candidates),
    len(df_pl_candidates)
)

'We have 14098 ส.ส. ทั้งหมด, (region 11181, partylist 2917)'

In [44]:
get_partyid_for_image(192)

81

In [45]:
df_zone_candidates[df_zone_candidates.PartyName == 'ไทยรักษาชาติ'].to_dict('record')

[{'Unnamed: 0': 7,
  'CandidateId': '985bd88b-c068-409c-a47f-742c8e003e01',
  'CandidateNo': 8,
  'FirstName': 'ทินวัฒน์',
  'LastName': 'เจียมอุย',
  'PartyName': 'ไทยรักษาชาติ',
  'PartyNumber': 77,
  'Title': 'นาย',
  'Votable': True,
  'province_id': 23,
  'province_name': 'ตราด',
  'zone_number': 1,
  'image': 'http://202.6.26.16/images/candidate/thumb/23/1/985bd88b-c068-409c-a47f-742c8e003e01.jpg',
  'name': 'ทินวัฒน์ เจียมอุย',
  'desc': 'ส.ส. เขต (ตราด เขต 1)',
  'memberType': 'zone'},
 {'Unnamed: 0': 50,
  'CandidateId': 'ed73a17e-0003-448f-8802-e392ae0fccf9',
  'CandidateNo': 5,
  'FirstName': 'เจษฎา',
  'LastName': 'โกลิบุตร',
  'PartyName': 'ไทยรักษาชาติ',
  'PartyNumber': 77,
  'Title': 'นาย',
  'Votable': True,
  'province_id': 82,
  'province_name': 'พังงา',
  'zone_number': 1,
  'image': 'http://202.6.26.16/images/candidate/thumb/82/1/ed73a17e-0003-448f-8802-e392ae0fccf9.jpg',
  'name': 'เจษฎา โกลิบุตร',
  'desc': 'ส.ส. เขต (พังงา เขต 1)',
  'memberType': 'zone'},
 {'Un

In [46]:
df_zone_candidates.loc[0].values

array([0, '82547da2-ee9e-4e55-a9e0-9aa414956075', 1, 'ไกรสีห์',
       'รัตนเศียร', 'ชาติพัฒนา', 45, 'นาย', True, 23, 'ตราด', 1,
       'http://202.6.26.16/images/candidate/thumb/23/1/82547da2-ee9e-4e55-a9e0-9aa414956075.jpg',
       'ไกรสีห์ รัตนเศียร', 'ส.ส. เขต (ตราด เขต 1)', 'zone'], dtype=object)

In [47]:
df_pl_candidates[df_pl_candidates.CandidateId == '00d3589a-aa2b-46fc-87c8-af7f11ff4346'].values

array([[2779, '00d3589a-aa2b-46fc-87c8-af7f11ff4346', 104, 'คิว',
        'อรุโณรส', 'พลังประชารัฐ', 192, 'นาย', True, 'คิว อรุโณรส',
        'ส.ส. บัญชีรายชื่อ (ลำดับที่ 104)',
        'http://202.6.26.16/images/candidate/thumb/00/81/00d3589a-aa2b-46fc-87c8-af7f11ff4346.jpg',
        'partylist', '-', '-']], dtype=object)

In [48]:
attributes = ['name', 'desc', 'image', 'PartyName', 'memberType', 'province_name', 'zone_number']

def extract_party_politician(party):
    zone = df_zone_candidates[df_zone_candidates.PartyName == party][attributes]
    plist = df_pl_candidates[df_pl_candidates.PartyName == party][attributes]

    df_party = pd.concat([zone, plist])
    df_party.to_csv('../data/party-politicians/%s.csv' % party.strip(), index=False, header=True)
    return df_party

In [49]:
# dd = extract_party_politician('พลังประชารัฐ')
# dd

In [50]:
for p in party_names:
    extract_party_politician(p)

In [51]:
with open ('../creden-scrape.txt', 'w') as f:
    f.write('\n'.join(party_names))

In [11]:
attributes = ['CandidateId', 'name', 'CandidateNo', 'PartyName', 'province_name', 'zone_number']

In [53]:
all_zone_candidates = df_zone_candidates[attributes].to_dict('records')

In [54]:
# df_zone_candidates
with open('../data/all-zone-candidates.json', 'w') as f:
    json.dump(all_zone_candidates, f, ensure_ascii=False)

# Check Missing files

In [12]:
party_names = list(set(df_zone_candidates.PartyName.values))

In [10]:
len(party_names)

81

In [16]:
for p in party_names:
    if not os.path.isfile('../creden/%s.csv.json' % p):
        print(p)

แผ่นดินธรรม
คนธรรมดาแห่งประเทศไทย
แทนคุณแผ่นดิน
ประชาธิปัตย์
สามัญชน 
ไทยธรรม
เพื่อแผ่นดิน
รักท้องถิ่นไทย
ผึ้งหลวง
คลองไทย
พลังไทยรักชาติ
ภูมิพลังเกษตรกรไทย
ประชาธิปไตยเพื่อประชาชน
พลังแรงงานไทย
ประชากรไทย
ชาติพัฒนา
พลังท้องถิ่นไท
ไทยรักษาชาติ 
ประชาชนปฏิรูป
พลังครูไทย
พลังปวงชนไทย


In [10]:
# print('\n'.join(party_names))

# Cross check numbers

In [12]:
df_all_candidates = pd.concat([df_zone_candidates[attributes], df_pl_candidates[attributes]])

In [13]:
pols_in_party_dict = df_all_candidates[['PartyName', 'CandidateNo']].groupby('PartyName').count().to_dict()['CandidateNo']

In [14]:
data_files = glob.glob("/Users/heytitle/projects/politician-business/app/assets/data/*.json")
print("We have %d data files" % len(data_files))
for f in data_files:
    partyName = f.split('/')[-1].split('.')[0].strip()
    print('Verifying %s' % partyName)
    with open(f) as json_file:  
        data = json.load(json_file)
        totalPoliticians = len(data)
        totalFromCSVs = pols_in_party_dict[partyName]
        assert totalPoliticians == totalFromCSVs, \
            '%s: %d politicians but  %d in CSVs' % (partyName, totalPoliticians, totalFromCSVs)

We have 81 data files
Verifying พลังไทยรักชาติ
Verifying รวมใจไทย
Verifying คลองไทย
Verifying เพื่อชาติ
Verifying พลังธรรมใหม่
Verifying ภาคีเครือข่ายไทย
Verifying พลังประชาธิปไตย
Verifying พลังไทสร้างชาติ
Verifying กรีน
Verifying พัฒนาประเทศไทย
Verifying พลังสังคม
Verifying ประชาภิวัฒน์
Verifying เศรษฐกิจใหม่
Verifying ประชาชาติ
Verifying พลังชาติไทย
Verifying เพื่อไทยพัฒนา
Verifying ประชานิยม
Verifying สังคมประชาธิปไตยไทย
Verifying เพื่อแผ่นดิน
Verifying พลังครูไทย
Verifying แทนคุณแผ่นดิน
Verifying สามัญชน
Verifying เพื่อนไทย
Verifying อนาคตใหม่
Verifying เพื่อคนไทย
Verifying ทางเลือกใหม่
Verifying ผึ้งหลวง
Verifying พลังไทยรักไทย
Verifying รักษ์ผืนป่าประเทศไทย
Verifying รวมพลังประชาชาติไทย
Verifying ประชาธรรมไทย
Verifying กสิกรไทย
Verifying เพื่อชีวิตใหม่
Verifying พลังศรัทธา
Verifying กลาง
Verifying ชาติพัฒนา
Verifying ไทรักธรรม
Verifying ประชากรไทย
Verifying มติประชา
Verifying พลังแรงงานไทย
Verifying ไทยธรรม
Verifying มหาชน
Verifying ภราดรภาพ
Verifying ความหวังใหม่
Verifying ไทยรุ

In [23]:
len(df_pl_candidates[df_pl_candidates.PartyName == 'ประชาธิปัตย์'])

300

# Verify duplicated politicians

In [36]:
data_files = glob.glob("/Users/heytitle/projects/politician-business/app/assets/data/*.json")
print("We have %d data files" % len(data_files))
for f in data_files:
    partyName = f.split('/')[-1].split('.')[0].strip()
#     print('Verifying %s' % partyName)
    with open(f) as json_file:  
        data = json.load(json_file)
        uniqPols = list(set(map(lambda x: x['name'], data)))
        totalUniqPoliticians = len(list(set(map(lambda x: x['name'], data))))
        totalPoliticians = len(data)
        if totalUniqPoliticians != totalPoliticians:
            print(data[:10])
#             print(uniqPols)
#             print(totalUniqPoliticians)
            print('We have dups politicians for %s, %d vs %d' % (partyName, totalUniqPoliticians, totalPoliticians) )

We have 78 data files
[{'name': 'ธีระ สลักเพชร', 'desc': 'ส.ส. เขต (ตราด เขต 1)', 'image': 'http://202.6.26.16/images/candidate/thumb/23/1/01a27143-b760-4056-aa16-1e18d1c1ff2a.jpg', 'PartyName': 'ประชาธิปัตย์', 'memberType': 'zone', 'province_name': 'ตราด', 'zone_number': '1', 'relatedTo': []}, {'name': 'สกลชัย อิสระเสนารักษ์', 'desc': 'ส.ส. เขต (นครนายก เขต 1)', 'image': 'http://202.6.26.16/images/candidate/thumb/26/1/46db98f7-2dde-4bf6-9383-5b48167e3726.jpg', 'PartyName': 'ประชาธิปัตย์', 'memberType': 'zone', 'province_name': 'นครนายก', 'zone_number': '1', 'relatedTo': [{'jptn': 'ห้างหุ้นส่วนจำกัด', 'JP_ENAME': 'CH.CHANCHAI SERVICE LIMITED PARTNERSHIP', 'OBJ_TNAME': 'การขายปลีกเชื้อเพลิงยานยนต์ในร้านค้าเฉพาะ สถานี ปั๊ม', '_id': '0263519000057', 'JP_TNAME': 'ช.ชาญชัย เซอร์วิส', 'JP_TYPE_CODE': '3', 'JP_NO': '0263519000057', 'PARTNER': ' นายสกลชัย อิสระเสนารักษ์', 'cpm': '400,000.00', 'stn': 'ยังดำเนินกิจการอยู่', 'activity_code': '47300'}, {'jptn': 'ห้างหุ้นส่วนจำกัด', 'JP_ENAME': 'S.

In [33]:
len(df_zone_candidates[df_zone_candidates['PartyName'] == 'ประชาธิปัตย์'])

350